<br>

# Introdução

- https://www.tjsp.jus.br/QuemSomos/QuemSomos/RegioesAdministrativasJudiciarias


In [ ]:
#!pip3 install traquitanas


In [ ]:
import pandas as pd
from bs4 import BeautifulSoup
from my_driver import Driver
from my_functions import find_text_between_parenthesis, keep_numbers
from paths import adds_path, driver_path, input_path, logs_path
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.options import Options as FirefoxOptions
from selenium.webdriver.firefox.service import Service as FirefoxService
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.wait import WebDriverWait
from traquitanas.scrapping import adds, gecko


<br>

## Drivers


In [ ]:
driver = Driver(
    my_driver_path=driver_path,
    my_logs_path=logs_path,
    verify_ssl=False,
)


In [ ]:
# Go TJSP
URL = 'https://www.tjsp.jus.br/QuemSomos/QuemSomos/RegioesAdministrativasJudiciarias'
driver.get(URL)
driver.add_extension_xpath(my_adds_path=adds_path)

# Windows
# driver.window_handles
# driver.switch_to.window(driver.window_handles[0])


<br>

## Juízes Diretores


In [ ]:
def get_rajs_info(raj_xpath):
    """
    Retorna dicionário das Circuncrições Judiciárias

    :param raj_xpath: selenium.element
    :type raj_xpath: _type_
    :return: _description_
    :rtype: dictionary
    """
    # Xpath
    raj_nome_xpath = raj_xpath.find_element(
        By.XPATH, './/p[@style="font-weight: bold"]'
    )
    diretor_xpath = raj_nome_xpath.find_element(
        By.XPATH, "./following-sibling::p"
    )
    email_xpath = diretor_xpath.find_element(By.XPATH, "./following-sibling::p")

    return {
        'raj_nome': raj_nome_xpath.text,
        'juiz_diretor': diretor_xpath.text,
        'email': email_xpath.text,
    }


In [ ]:
# Localiza <div> irmã dos "Juízes Diretores"
juiz_xpath = driver.find_element(By.XPATH, '//*[text()="Juízes Diretores"]//..')
print(juiz_xpath.text)


In [ ]:
# RAJs
rajs_xpath = juiz_xpath.find_element(By.XPATH, './following-sibling::div')

# Comarcas (linhas)
comarcas_xpath = rajs_xpath.find_elements(
    By.XPATH, './/li[@class="list-group-item"]'
)
print(len(comarcas_xpath))


In [ ]:
list_comarcas = []
for comarca_xpath in comarcas_xpath:
    # Comarca
    print(comarca_xpath.text)

    # Procura
    raj_xpath = comarca_xpath.find_element(
        By.XPATH, './../preceding-sibling::div'
    )

    # CJs
    dict_circuncricao = None
    dict_circuncricao = get_rajs_info(raj_xpath)
    dict_circuncricao.update({'comarca_cirscunscricao': comarca_xpath.text})

    # Lista de Comarcas
    list_comarcas.append(dict_circuncricao)


In [ ]:
# Results
list_comarcas[:2]


In [ ]:
# Create Table
df = pd.DataFrame(list_comarcas)

# Results
df.info()
df.head()


<br>

## Adjust Table


In [ ]:
# Adjust Data
df['raj_nome'] = df['raj_nome'].str.replace('–', '-')
df[['raj_sigla', 'raj_regiao']] = df['raj_nome'].str.split(' - ', expand=True)
df[['comarca', 'cj_sigla']] = df['comarca_cirscunscricao'].str.split(
    ' - ', expand=True
)

# Números
df['raj_id'] = df['raj_sigla'].apply(lambda x: keep_numbers(x))
df['cj_id'] = df['cj_sigla'].apply(lambda x: keep_numbers(x))
df['cj_nome'] = df['cj_sigla'].replace(
    'CJ', 'Circunscrição Judiciária', regex=True
)

# Prefixo
df['juiz_diretor'] = (
    df['juiz_diretor']
    .replace('Juiza diretora: ', '', regex=True)
    .replace('Juiz diretor: ', '', regex=True)
)
df['email'] = df['email'].apply(lambda x: find_text_between_parenthesis(x))

# Results
df.info()
df.head()


In [ ]:
list(df.columns)


In [ ]:
df = df[
    [
        # RAJ
        'raj_id',
        'raj_nome',
        'raj_sigla',
        'raj_regiao',
        'juiz_diretor',
        'email',
        # CJ
        'cj_id',
        'cj_nome',
        'cj_sigla',
        # Comarca
        'comarca',
    ]
]

# Results
df.info()
df.head()


<br>

## Results


<br>

### RAJs


In [ ]:
df_raj = df[
    [
        # RAJ
        'raj_id',
        'raj_nome',
        'raj_sigla',
        'raj_regiao',
        'juiz_diretor',
        'email',
    ]
]

df_raj = df_raj.drop_duplicates()
df_raj = df_raj.sort_values(by='raj_id')
df_raj = df_raj.reset_index(drop=True)
display(df_raj.head())

df_raj.to_csv(
    input_path / 'Dim_RAJs.csv',
    index=False,
)

df_raj.to_excel(
    input_path / 'Dim_RAJs.xlsx',
    index=False,
    sheet_name='Dim_RAJs',
)


<br>

### CJs


In [ ]:
df_cjs = df[
    [
        # CJ
        'cj_id',
        'cj_nome',
        'cj_sigla',
        'raj_id',
    ]
]

df_cjs.loc[df_cjs['cj_id'] == '', 'cj_id'] = '0'
df_cjs['cj_id'] = df_cjs['cj_id'].copy()
df_cjs['cj_id'] = df_cjs['cj_id'].astype('int')
df_cjs = df_cjs.drop_duplicates()
df_cjs = df_cjs.sort_values(by='cj_id')
df_cjs = df_cjs.reset_index(drop=True)
display(df_cjs.head())

df_cjs.to_csv(
    input_path / 'Dim_CircunscricaoJudiciaria.csv',
    index=False,
)

df_cjs.to_excel(
    input_path / 'Dim_CircunscricaoJudiciaria.xlsx',
    index=False,
    sheet_name='Dim_CircunscricaoJudiciaria',
)


<br>

### Comarcas


In [ ]:
df_comarca = df[
    [
        # Comarca
        'comarca',
        'cj_id',
    ]
]

df_comarca.loc[df_comarca['cj_id'] == '', 'cj_id'] = '0'
df_comarca['cj_id'] = df_comarca['cj_id'].copy()
df_comarca['cj_id'] = df_comarca['cj_id'].astype('int')
df_comarca = df_comarca.drop_duplicates()

df_comarca = df_comarca.rename(
    {
        'comarca': 'comarca_nome_tjsp',
    },
    axis=1,
)

df_comarca = df_comarca.sort_values(by='comarca_nome_tjsp')
df_comarca = df_comarca.iloc[
    df_comarca['comarca_nome_tjsp'].str.normalize('NFKD').argsort()
]
df_comarca = df_comarca.reset_index(drop=True)
display(df_comarca.head())

df_comarca.to_csv(
    input_path / 'Dim_Comarcas_CJs.csv',
    index=False,
)

df_comarca.to_excel(
    input_path / 'Dim_Comarcas_CJs.xlsx',
    index=False,
    sheet_name='Dim_Comarcas_CJs',
)


In [ ]:
comarcas = df['comarca']

print(f'São {len(comarcas)} comarcas')
print(f'São {len(set(comarcas))} comarcas')


In [ ]:
driver.quit()
